In [1]:
import pandas as pd
import numpy as np
import torch
import seaborn as sns
from sklearn.model_selection import train_test_split

In [2]:
train = pd.read_csv('train_processed.csv',parse_dates=['datetime'], index_col='datetime')
test = pd.read_csv('test_processed.csv', parse_dates=['datetime'], index_col='datetime')

In [3]:
X = train.drop(['count'], axis=1).values
y = train['count'].values
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

In [4]:
X_train = torch.tensor(X_train, dtype=torch.float)
X_val = torch.tensor(X_val, dtype=torch.float)
y_train = torch.tensor(y_train, dtype=torch.float)
y_val = torch.tensor(y_val, dtype=torch.float)

In [5]:
class Net(torch.nn.Module):
    def __init__(self, n_feature, n_hidden, n_output):
        super(Net, self).__init__()
        self.hidden = torch.nn.Linear(n_feature, n_hidden)
        self.output = torch.nn.Linear(n_hidden, n_output)

    def forward(self, x):
        x = torch.relu(self.hidden(x))
        x = self.output(x)
        return x.squeeze()

n_feature, n_hidden, n_output = len(X_train[0]), 32, 1

net = Net(n_feature, n_hidden, n_output)

In [6]:
# 優化器
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(net.parameters(), lr = 0.01)

In [7]:
for epoch in range(100):
    optimizer.zero_grad()
    y_pred = net(X_train)
    loss = criterion(y_pred, y_train)
    loss.backward()
    optimizer.step()

    # Print the loss every 10 epochs
    if (epoch+1) % 10 == 0:
        with torch.no_grad():
            y_val_pred = net(X_val)
            val_loss = criterion(y_val_pred, y_val)
            print(f'Epoch {epoch+1}, Training loss: {loss.item()}, Validation loss: {val_loss.item()}')
    else:
        pass

Epoch 10, Training loss: 25384.4375, Validation loss: 23721.55859375
Epoch 20, Training loss: 15927.6875, Validation loss: 15098.435546875
Epoch 30, Training loss: 7477.853515625, Validation loss: 7007.41845703125
Epoch 40, Training loss: 1627.1676025390625, Validation loss: 1225.90478515625
Epoch 50, Training loss: 136.18792724609375, Validation loss: 220.12596130371094
Epoch 60, Training loss: 364.6963195800781, Validation loss: 363.91497802734375
Epoch 70, Training loss: 87.2016830444336, Validation loss: 79.11560821533203
Epoch 80, Training loss: 42.82836151123047, Validation loss: 54.38473892211914
Epoch 90, Training loss: 42.56136703491211, Validation loss: 47.56053161621094
Epoch 100, Training loss: 22.874711990356445, Validation loss: 27.316774368286133



TODO : 做test的預測和提交kaggle

> https://towardsdatascience.com/how-to-organize-your-data-science-project-dd6599cf000a